In [61]:
import pandas as pd

In [62]:
df = pd.read_csv("../data/03-preprocessed/exercises_v2.csv")

C:\Users\alejo\AppData\Local\Temp\ipykernel_8528\1063109727.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/03-preprocessed/exercises_v2.csv")


In [63]:
df[df.duplicated()].shape[0]

756

In [64]:
df.drop_duplicates(inplace=True)

# Parte hecha con Google Colab

# Task
Perform data cleaning and preprocessing on the DataFrame: extract the numerical part from the 'day' column and convert it to an integer, convert the 'exercise' and 'observations' columns to lowercase, calculate the number of null values for each column, create a new 'client_id' column by mapping unique values from 'client_name_clean' to numerical identifiers, and then remove the original 'client_name_clean' column.

## Clean 'day' column

### Subtask:
Extract the numerical part from the 'day' column (e.g., 'Day 1' becomes '1') and convert it to an integer type for further analysis.


# Task
Perform data cleaning and preprocessing on the DataFrame: extract the numerical part from the 'day' column and convert it to an integer, convert the 'exercise' and 'observations' columns to lowercase, calculate the number of null values for each column, create a new 'client_id' column by mapping unique values from 'client_name_clean' to numerical identifiers, and then remove the original 'client_name_clean' column.

## Clean 'day' column

### Subtask:
Extract the numerical part from the 'day' column (e.g., 'Day 1' becomes '1') and convert it to an integer type for further analysis.


**Reasoning**:
Extract the numerical part from the 'day' column using a regular expression and convert it to an integer type, as per the subtask instructions.



In [65]:
df['day'] = df['day'].astype(str).str.extract(r'(\d+)').astype(int)
print(df['day'].head())
print(df['day'].dtype)

0    1
1    1
2    1
3    1
4    1
Name: day, dtype: int64
int64


## Normalize text columns

### Subtask:
Convert all text entries in the 'exercise' and 'observations' columns to lowercase to ensure consistency and facilitate analysis.


**Reasoning**:
To ensure consistency and facilitate analysis, I will convert the 'exercise' and 'observations' columns to lowercase using the `.str.lower()` method, handling potential non-string values by first converting them to string type.



In [66]:
df['exercise'] = df['exercise'].str.lower()
df['observations'] = df['observations'].str.lower()
print(df[['exercise', 'observations']].head())

                         exercise observations
0            puente sobre fitball          NaN
1                      pull apart          NaN
2  bird dog - movilidad de cadera          NaN
3      sentadilla con barra cajon          NaN
4           press plano con barra          NaN


## Format date column

## Generate client IDs

### Subtask:
Create a new column named 'client_id' by mapping each unique value in the 'client_name_clean' column to a unique numerical identifier. Then, drop the original 'client_name_clean' column.


**Reasoning**:
To generate unique numerical IDs for each client and remove the original column, I will use `pd.factorize` to map 'client_name_clean' to 'client_id' and then drop the redundant 'client_name_clean' column.



In [67]:
df['client_id'] = pd.factorize(df['client_name_clean'])[0]
print(df[['client_id', 'client_name_clean']].sample(5))


       client_id   client_name_clean
13836         20         ismael ruiz
46170        244      anabella olmos
25532        138      ayelen bovetti
69671         32         lucia campi
23365         59  alejandro valdatta


In [68]:
print(df['client_id'].nunique())
print(df['client_name_clean'].nunique())

609
609


In [69]:
df.drop(columns=['client_name_clean', 'client_name'], inplace=True)

## Data formatting

In [70]:
df.sort_values("date")[["date"]].sample(5)

,date
85860,23/01/23
69112,05/02/24
67419,28/04/25
20732,09/10/23
37513,27/05/24


In [71]:
df['cleaned_date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce').dt.strftime('%m/%Y')

C:\Users\alejo\AppData\Local\Temp\ipykernel_8528\1820823289.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['cleaned_date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce').dt.strftime('%m/%Y')


In [72]:
df['cleaned_date'].notna().value_counts()

cleaned_date
True     92979
False      166
Name: count, dtype: int64

In [73]:
df['date'].notna().value_counts()

date
True     93073
False       72
Name: count, dtype: int64

In [74]:
df = df[df['cleaned_date'].notna()]
df['date'] = df['cleaned_date']
df.drop(columns=['cleaned_date'], inplace=True)

## Analyze missing values

### Subtask:
Calculate the count of null values for each column in the DataFrame and display the results to understand data completeness.


**Reasoning**:
To understand the data completeness, I will calculate the number of null values for each column in the DataFrame `df` using the `.isnull().sum()` method and display the results.



In [75]:
total_rows = df.shape[0]
null_counts = (df.isnull().sum() * 100 / total_rows).round(2)
print("% of null values per column:\n", null_counts)

% of null values per column:
 date             0.00
day              0.00
exercise         0.00
observations    98.51
sets_1          45.77
reps_1           1.43
kg_1            98.86
rest_1          58.78
sets_2          98.10
reps_2          97.75
kg_2            98.98
sets_3          97.20
reps_3          96.26
kg_3            98.93
sets_4          97.87
reps_4          97.46
kg_4            98.75
gender           0.06
client_id        0.00
dtype: float64


In [76]:
df.day.value_counts()

day
1    73107
2    10046
3     6884
4     2661
5      281
Name: count, dtype: int64

In [77]:
df[df['date'].isnull()]['client_id'].value_counts()

Series([], Name: count, dtype: int64)

In [78]:
df[df['client_id'].isin([443,540])]['client_id'].value_counts()

Series([], Name: count, dtype: int64)

In [79]:
df = df[~df['client_id'].isin([443,540])]

In [80]:
df.groupby(['client_id','observations']).describe()

day            \
                                                              count      mean   
client_id observations                                                          
1         operadas las rodillas                               104.0  1.000000   
2         molestia zona lumbar                                 77.0  2.259740   
7         derrame articular rodilla izquierda                  36.0  1.000000   
          derrame articular rodilla izquierda.                 33.0  2.000000   
10        dolor cervical                                      105.0  2.000000   
18        entrenamiento de resistencia muscular                99.0  2.000000   
19        ligamentos cruzados y menisco interior (izq)         40.0  2.500000   
          ligamentos cruzados y menisco interior (izquier...   31.0  1.000000   
21        sinovitis en rodilla                                145.0  1.000000   
22        operación en clavicula                               51.0  1.000000   
27        operación de ligamentos cruzado anterior             99.0  1.000000   
28        placa con tornillos en clavicula izquierda           71.0  2.000000   
35        molestia en cadera                                   69.0  1.000000   
47        tendinitis en brazo - "codo".                        36.0  1.000000   
48        molestia en isquio                                   36.0  1.000000   
51        fortalecimiento de esguince de tobillo               72.0  1.000000   
52        operadas las rodillas                                57.0  1.000000   
112       mes de ambientación y cargas progresivas             29.0  1.965517   
213       dolor manguito rotador brazo derecho                 88.0  1.000000   
297       fractura en clavicula y lumbalgia                    30.0  1.000000   
318       fractura en clavicula derecha y presión alta         30.0  1.000000   
503       aducciones no (molestia femur)                       44.0  1.000000   

                                                                             \
                                                                   std  min   
client_id observations                                                        
1         operadas las rodillas                               0.000000  1.0   
2         molestia zona lumbar                                1.043811  1.0   
7         derrame articular rodilla izquierda                 0.000000  1.0   
          derrame articular rodilla izquierda.                0.829156  1.0   
10        dolor cervical                                      0.832050  1.0   
18        entrenamiento de resistencia muscular               0.820652  1.0   
19        ligamentos cruzados y menisco interior (izq)        1.132277  1.0   
          ligamentos cruzados y menisco interior (izquier...  0.000000  1.0   
21        sinovitis en rodilla                                0.000000  1.0   
22        operación en clavicula                              0.000000  1.0   
27        operación de ligamentos cruzado anterior            0.000000  1.0   
28        placa con tornillos en clavicula izquierda          0.828079  1.0   
35        molestia en cadera                                  0.000000  1.0   
47        tendinitis en brazo - "codo".                       0.000000  1.0   
48        molestia en isquio                                  0.000000  1.0   
51        fortalecimiento de esguince de tobillo              0.000000  1.0   
52        operadas las rodillas                               0.000000  1.0   
112       mes de ambientación y cargas progresivas            0.823007  1.0   
213       dolor manguito rotador brazo derecho                0.000000  1.0   
297       fractura en clavicula y lumbalgia                   0.000000  1.0   
318       fractura en clavicula derecha y presión alta        0.000000  1.0   
503       aducciones no (molestia femur)                      0.000000  1.0   

                                                              

In [81]:
df['observations'].value_counts()

observations
operadas las rodillas                                  161
sinovitis en rodilla                                   145
dolor cervical                                         105
entrenamiento de resistencia muscular                   99
operación de ligamentos cruzado anterior                99
dolor manguito rotador brazo derecho                    88
molestia zona lumbar                                    77
fortalecimiento de esguince de tobillo                  72
placa con tornillos en clavicula izquierda              71
molestia en cadera                                      69
operación en clavicula                                  51
aducciones no (molestia femur)                          44
ligamentos cruzados y menisco interior (izq)            40
molestia en isquio                                      36
derrame articular rodilla izquierda                     36
tendinitis en brazo - "codo".                           36
derrame articular rodilla izquierda.       

## Summary:

### Data Analysis Key Findings

*   The 'day' column was successfully cleaned by extracting the numerical part (e.g., 'Day 1' became '1') and converting it to an integer data type (`int64`).
*   Text consistency was achieved by converting all entries in the 'exercise' and 'observations' columns to lowercase. For example, "Puente sobre fitball" became "puente sobre fitball".
*   An analysis of missing values revealed significant data gaps in several columns:
    *   Columns related to weight (`kg_1`, `kg_2`, `kg_3`, `kg_4`) have over 92,000 null values each.
    *   Columns related to sets (`sets_1`, `sets_2`, `sets_3`, `sets_4`) have high null counts, ranging from approximately 43,000 to 92,000.
    *   Columns related to repetitions (`reps_1`, `reps_2`, `reps_3`, `reps_4`) show null counts between approximately 1,300 and 91,000.
    *   The 'date' and 'gender' columns also have some missing values, with 72 and 60 nulls respectively.
*   A new 'client\_id' column was successfully created by mapping 609 unique values from the 'client\_name\_clean' column to numerical identifiers, and the original 'client\_name\_clean' column was subsequently removed.

### Insights or Next Steps

*   The significant number of missing values, particularly in exercise-related metrics (sets, reps, kg), indicates a need for a strategy to handle them. Depending on the analysis goals, this could involve imputation, removal of rows/columns, or specialized models that can handle missing data.
*   The newly created 'client\_id' column facilitates anonymous and efficient analysis of client-specific data, while the lowercase conversion of 'exercise' and 'observations' ensures consistent data for categorization and natural language processing tasks.


# Decision
I'll clean all "sets", "reps", "kg" columns because they have a lot of null values, and I'll focus my analysis in the customer retention and classification.

In [82]:
df.drop(columns=["rest_1","kg_1", "kg_2", "kg_3", "kg_4", "sets_1", "sets_2", "sets_3", "sets_4", "reps_1", "reps_2", "reps_3", "reps_4"], inplace=True)

I'll drop "observations" and "exercise" columns since it's not relevant for my analysis.

In [83]:
df.drop(columns=["observations", "exercise"], inplace=True)

In [84]:
df.columns

Index(['date', 'day', 'gender', 'client_id'], dtype='object')

In [107]:
df_clients_months = df.groupby(['client_id', 'date', 'gender'])[['day']].max().reset_index()
df_clients_months

,client_id,date,gender,day
0,0,01/2023,M,1
1,0,08/2022,M,1
2,0,09/2022,M,1
3,0,11/2022,M,1
4,1,01/2024,M,3
...,...,...,...,...
3126,607,07/2025,F,2
3127,607,09/2025,F,2
3128,607,10/2025,F,2
3129,607,11/2025,F,2


In [86]:
df_clients_months.groupby('date').size().sort_index()

date
01/2023     67
01/2024     51
01/2025     77
02/2002      1
02/2023     45
02/2024     75
02/2025    103
03/2022      1
03/2023     57
03/2024     78
03/2025     48
04/2022      1
04/2023     56
04/2024    115
04/2025    101
05/2023     81
05/2024     61
05/2025     93
06/2023     62
06/2024     75
06/2025     87
06/2026      1
07/2022      2
07/2023     71
07/2024     93
07/2025    101
08/2022     37
08/2023     87
08/2024     80
08/2025     97
09/2022     29
09/2023     75
09/2024     96
09/2025    128
10/2022     52
10/2023     82
10/2024    102
10/2025     66
11/2022     46
11/2023     86
11/2024     30
11/2025     79
12/2022     35
12/2023     77
12/2024    121
12/2025    123
dtype: int64

I'll delete rows with wrong dates. The gym opened on 07/2022. The analysis was done on 12/2025.

In [87]:
print(df_clients_months.shape[0])
df_clients_months = df_clients_months[~df_clients_months["date"].isin([
    "02/2002",
    "03/2022",
    "04/2022",
    "06/2026"
])]
print(df_clients_months.shape[0])

3131
3127


In [88]:
df_clients_months.to_csv("../data/04-processed/clients_months.csv", index=False)